In [1]:
import pandas as pd
import numpy as np
import warnings
from Dataset_functions import *
# Suppress all warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [551]:
teams = season_data['HomeTeam'].unique()
    # teams_pairs = [[i,j]  for i in teams for j in teams if i!=j]
teams_pairs = [sorted([i, j]) for i in teams for j in teams if i != j]

# Remove duplicates where the same pair appears in reverse order
distinct_teams_pairs = []
for pair in teams_pairs:
    if pair not in distinct_teams_pairs:
        distinct_teams_pairs.append(pair)
teams_pairs = distinct_teams_pairs

In [563]:
def get_team_h2h(league_data):
    league_data['Home_h2h_Goals'] = np.zeros(len(league_data))
    league_data['Home_h2h_Points'] = np.zeros(len(league_data))
    league_data['Away_h2h_Goals'] = np.zeros(len(league_data))
    league_data['Away_h2h_Points'] = np.zeros(len(league_data))
    teams = league_data['HomeTeam'].unique()
    teams_pairs = [[i,j]  for i in teams for j in teams if i!=j]
    teams_pairs = [sorted([i, j]) for i in teams for j in teams if i != j]

    # Remove duplicates where the same pair appears in reverse order
    distinct_teams_pairs = []
    for pair in teams_pairs:
        if pair not in distinct_teams_pairs:
            distinct_teams_pairs.append(pair)
    teams_pairs = distinct_teams_pairs
    for k in range(len(teams_pairs)):
        team_stats = {}
        teams_h2h = league_data.loc[((league_data['HomeTeam'] == teams_pairs[k][0]) &(league_data['AwayTeam'] == teams_pairs[k][1])) | ((league_data['AwayTeam'] == teams_pairs[k][0]) &(league_data['HomeTeam'] == teams_pairs[k][1]))]
        
        def get_team_stats(team):
            if team not in team_stats:
                team_stats[team] = {'goals': 0, 'points': 0}
            return team_stats[team]['goals'], team_stats[team]['points']
        
        # Function to update the cumulative goals and points for a team
        def update_team_stats(team, goals, points):
            if team not in team_stats:
                team_stats[team] = {'goals': 0, 'points': 0}
            team_stats[team]['goals'] += goals
            team_stats[team]['points'] += points
        
        # Lists to store cumulative data before each match
        home_team_cum_goals = []
        home_team_cum_points = []
        away_team_cum_goals = []
        away_team_cum_points = []
        
        
        for _, row in teams_h2h.iterrows():
            home_team = row['HomeTeam']
            away_team = row['AwayTeam']
            
            # Get cumulative stats for both teams before the match
            home_goals_before, home_points_before = get_team_stats(home_team)
            away_goals_before, away_points_before = get_team_stats(away_team)
            
            # Append the current cumulative stats to the lists
            home_team_cum_goals.append(home_goals_before)
            home_team_cum_points.append(home_points_before)
            away_team_cum_goals.append(away_goals_before)
            away_team_cum_points.append(away_points_before)
            
            # Update the stats after the match
            # Goals
            home_goals = row['FTHG']
            away_goals = row['FTAG']
            update_team_stats(home_team, home_goals, 0)
            update_team_stats(away_team, away_goals, 0)
            
            # Points
            if home_goals > away_goals:
                update_team_stats(home_team, 0, 3)  # Home team wins
            elif home_goals < away_goals:
                update_team_stats(away_team, 0, 3)  # Away team wins
            else:
                update_team_stats(home_team, 0, 1)  # Draw
                update_team_stats(away_team, 0, 1)  # Draw
        # print(home_team_cum_goals)
        # fixtures = np.linspace(0,len(home_team_cum_goals)-1,len(home_team_cum_goals))
        teams_h2h['Home_h2h_Goals'] = home_team_cum_goals/np.linspace(0,len(home_team_cum_goals)-1,len(home_team_cum_goals))
        teams_h2h['Home_h2h_Points'] = home_team_cum_points/np.linspace(0,len(home_team_cum_goals)-1,len(home_team_cum_goals))
        teams_h2h['Away_h2h_Goals'] = away_team_cum_goals/np.linspace(0,len(home_team_cum_goals)-1,len(home_team_cum_goals))
        teams_h2h['Away_h2h_Points'] = away_team_cum_points/np.linspace(0,len(home_team_cum_goals)-1,len(home_team_cum_goals))
        teams_h2h.fillna(0,inplace = True)
        league_data.update(teams_h2h)
    league_data.replace([np.inf, -np.inf], 0, inplace=True)
        # league_data = pd.merge(league_data, teams_h2h[['Date',	'HomeTeam',	'AwayTeam','Home_h2h_Goals', 'Home_h2h_Points', 'Away_h2h_Goals', 'Away_h2h_Points']], on =['Date', 'HomeTeam', 'AwayTeam'], how = 'outer')
    return league_data

In [607]:
premier_league_season17_19 = prepare_dataframe('17_18_eng.csv', '2017-2018')
premier_league_season18_19 = prepare_dataframe('18_19_eng.csv', '2018-2019')
premier_league_season19_20 = prepare_dataframe('19_20_eng.csv', '2019-2020')
premier_league_season20_21 = prepare_dataframe('20_21_eng.csv', '2020-2021')
premier_league_season21_22 = prepare_dataframe('21_22_eng.csv', '2021-2022')
premier_league_season22_23 = prepare_dataframe('22_23_eng.csv', '2022-2023')
premier_league_season23_24 = prepare_dataframe('23_24_eng.csv', '2023-2024')
premier_league_season24_25 = prepare_dataframe('24_25_eng.csv', '2024-2025')

premier_league_data = pd.concat([premier_league_season17_19, premier_league_season18_19, premier_league_season19_20, premier_league_season20_21, premier_league_season21_22, premier_league_season22_23, premier_league_season23_24,premier_league_season24_25])
premier_league_data.fillna(0, inplace = True)
# premier_league_data[['Home_h2h_Goals', 'Home_h2h_Points', 'Away_h2h_Goals', 'Away_h2h_Points']] = season_data[['Home_h2h_Goals', 'Home_h2h_Points', 'Away_h2h_Goals', 'Away_h2h_Points']]
premier_league_data.to_csv('premier_league_data.csv', index = False)

In [603]:
# premier_league_data.reset_index(drop = True, inplace = True)
# premier_league_data[['Home_h2h_Goals', 'Home_h2h_Points', 'Away_h2h_Goals', 'Away_h2h_Points']] = season_data[['Home_h2h_Goals', 'Home_h2h_Points', 'Away_h2h_Goals', 'Away_h2h_Points']]
# premier_league_data.to_csv('premier_league_data.csv', index = False)

In [3]:
def prepare_dataframe_championship(csv_file, season):
    season_data = pd.read_csv(csv_file)
    teams = season_data['HomeTeam'].unique()
    url = f"https://fbref.com/en/comps/10/{season}/schedule/{season}-Championship-Scores-and-Fixtures"
    season_data = join_scraped(url, season_data, season, teams, '10')
    season_data = clean_v2(season_data)
    season_data = create_seasonal_table(season_data,teams)
    season_data = calculate_odds(season_data)
    return season_data

In [4]:
championship_season18_19 = prepare_dataframe_championship('championship_18_19.csv', '2018-2019')
championship_season19_20 = prepare_dataframe_championship('championship_19_20.csv', '2019-2020')
championship_season20_21 = prepare_dataframe_championship('championship_20_21.csv', '2020-2021')
championship_season21_22 = prepare_dataframe_championship('championship_21_22.csv', '2021-2022')
championship_season22_23 = prepare_dataframe_championship('championship_22_23.csv', '2022-2023')
championship_season23_24 = prepare_dataframe_championship('championship_23_24.csv', '2023-2024')
championship_season24_25 = prepare_dataframe_championship('championship_24_25.csv', '2024-2025')

In [6]:
championship_data = pd.concat([championship_season18_19, championship_season19_20, championship_season20_21, championship_season21_22, championship_season22_23, championship_season23_24,championship_season24_25])
championship_data.fillna(0, inplace = True)
championship_data.to_csv('championship_data.csv', index = False)

In [7]:
england_data = pd.concat([premier_league_data, championship_data])

In [8]:
england_data.fillna(0, inplace = True)

In [9]:
england_data.sort_values(['Date'], ascending = True, inplace = True)

In [10]:
england_data.reset_index(inplace= True, drop = True)

In [37]:
def prepare_dataframe_spanish1(csv_file, season):
    season_data = pd.read_csv(csv_file)
    teams = season_data['HomeTeam'].unique()
    url = f"https://fbref.com/en/comps/12/{season}/schedule/{season}-La-Liga-Scores-and-Fixtures"
    season_data = join_scraped(url, season_data, season, teams, '12')
    season_data = clean_v2(season_data)
    season_data = create_seasonal_table(season_data,teams)
    season_data = calculate_odds(season_data)
    return season_data

def clean_seasonal_data(dataset):
    dataset.drop(['HTHG','HTAG','HTR','Referee', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA','AvgH','AvgD',	'AvgA',	'B365>2.5',	'B365<2.5',	'P>2.5',	'P<2.5',	'Max>2.5',	'Max<2.5',	'Avg>2.5',	'Avg<2.5',	'AHh',	'B365AHH',	'B365AHA',	'PAHH',	'PAHA',	'MaxAHH',	'MaxAHA',	'AvgAHH',	'AvgAHA',	'B365CH',	'B365CD',	'B365CA',	'BWCH',	'BWCD',	'BWCA',	'IWCH',	'IWCD',	'IWCA',	'PSCH',	'PSCD',	'PSCA',	'WHCH',	'WHCD',	'WHCA',	'VCCH',	'VCCD',	'VCCA',	'MaxCH',	'MaxCD',	'MaxCA',	'AvgCH',	'AvgCD',	'AvgCA',	'B365C>2.5',	'B365C<2.5',	'PC>2.5',	'PC<2.5',	'MaxC>2.5',	'MaxC<2.5',	'AvgC>2.5',	'AvgC<2.5',	'AHCh',	'B365CAHH',	'B365CAHA',	'PCAHH',	'PCAHA',	'MaxCAHH',	'MaxCAHA',	'AvgCAHH',	'AvgCAHA','BFH',	'BFD',	'BFA',	'1XBH',	'1XBD',	'1XBA',	'BFEH',	'BFED',	'BFEA',	'BFE>2.5',	'BFE<2.5',	'BFEAHH',	'BFEAHA',	'BFCH',	'BFCD',	'BFCA'	'1XBCH',	'1XBCD',	'1XBCA',	'BFECH',	'BFECD',	'BFECA',	'BFEC>2.5',	'BFEC<2.5',	'BFECAHH',	'BFECAHA'	 ], inplace =True, axis = 1, errors='ignore')
    dataset = pd.get_dummies(dataset, columns=['HomeTeam', 'AwayTeam'])
    dummy_columns = dataset.select_dtypes(bool).columns
    dataset[dummy_columns] = dataset[dummy_columns].astype(int)
    dataset['Date'] = pd.to_datetime(dataset['Date'],format = '%d/%m/%Y' )
    return dataset

def join_scraped(url, season_data, season, teams, league_identifier):
    xgoals= scrape_data(url, season, league_identifier)
    xgoals = xgoals[['Home Team', 'Away Team','Home xG', 'Away xG']]

    teams = sorted(teams)
    teams2 = sorted(xgoals['Home Team'].unique())

    dict_teams = {}
    for i in range(len(teams2)):
        dict_teams[teams2[i]] = teams[i]
    xgoals = xgoals.replace(dict_teams) 
    season_data = pd.merge(season_data, xgoals, left_on = ['HomeTeam', 'AwayTeam'], right_on = ['Home Team', 'Away Team'])
    season_data['Home xG'] = pd.to_numeric(season_data['Home xG'], errors='coerce')
    season_data['Away xG'] = pd.to_numeric(season_data['Away xG'], errors='coerce')
    xgoals = season_data[['Home xG','Away xG']]
    season_data = clean_seasonal_data(season_data)
    season_data[['Home xG','Away xG']] = xgoals
    return season_data

In [ ]:
# spanish1_season17_18 = prepare_dataframe_spanish1('SP1_17_18.csv', '2017-2018')
spanish1_season18_19 = prepare_dataframe_spanish1('SP1_18_19.csv', '2018-2019')
spanish1_season19_20 = prepare_dataframe_spanish1('SP1_19_20.csv', '2019-2020')
spanish1_season20_21 = prepare_dataframe_spanish1('SP1_20_21.csv', '2020-2021')
spanish1_season21_22 = prepare_dataframe_spanish1('SP1_21_22.csv', '2021-2022')
spanish1_season22_23 = prepare_dataframe_spanish1('SP1_22_23.csv', '2022-2023')
spanish1_season23_24 = prepare_dataframe_spanish1('SP1_23_24.csv', '2023-2024')
spanish1_season24_25 = prepare_dataframe_spanish1('SP_24_25.csv', '2024-2025')

In [43]:
spanish1_data = pd.concat([spanish1_season18_19, spanish1_season19_20, spanish1_season20_21, spanish1_season21_22, spanish1_season22_23, spanish1_season23_24,spanish1_season24_25])
spanish1_data.fillna(0, inplace = True)
spanish1_data.to_csv('spanish1_data.csv', index = False)

In [55]:
def prepare_dataframe_spanish2(csv_file, season):
    season_data = pd.read_csv(csv_file)
    teams = season_data['HomeTeam'].unique()
    url = f"https://fbref.com/en/comps/17/{season}/schedule/{season}-Segunda-Division-Scores-and-Fixtures"
    season_data = join_scraped(url, season_data, season, teams, '17')
    season_data = clean_v2(season_data)
    season_data = create_seasonal_table(season_data,teams)
    season_data = calculate_odds(season_data)
    return season_data

In [57]:
spanish2_season18_19 = prepare_dataframe_spanish2('SP2_18_19.csv', '2018-2019')
spanish2_season19_20 = prepare_dataframe_spanish2('SP2_19_20.csv', '2019-2020')
spanish2_season20_21 = prepare_dataframe_spanish2('SP2_20_21.csv', '2020-2021')
spanish2_season21_22 = prepare_dataframe_spanish2('SP2_21_22.csv', '2021-2022')
spanish2_season22_23 = prepare_dataframe_spanish2('SP2_22_23.csv', '2022-2023')
spanish2_season23_24 = prepare_dataframe_spanish2('SP2_23_24.csv', '2023-2024')
# spanish2_season24_25 = prepare_dataframe_spanish2('SP2_24_25.csv', '2024-2025')

IndexError: list index out of range

In [60]:
spanish2_data = pd.concat([spanish2_season18_19, spanish2_season19_20, spanish2_season20_21, spanish2_season21_22, spanish2_season22_23, spanish2_season23_24])
spanish2_data.fillna(0, inplace = True)
spanish2_data.to_csv('spanish2_data.csv', index = False)

In [62]:
def prepare_dataframe_italian1(csv_file, season):
    season_data = pd.read_csv(csv_file)
    teams = season_data['HomeTeam'].unique()
    url = f"https://fbref.com/en/comps/11/{season}/schedule/{season}-Serie-A-Scores-and-Fixtures"
    season_data = join_scraped(url, season_data, season, teams, '11')
    season_data = clean_v2(season_data)
    season_data = create_seasonal_table(season_data,teams)
    season_data = calculate_odds(season_data)
    return season_data

In [64]:
# spanish1_season17_18 = prepare_dataframe_spanish1('SP1_17_18.csv', '2017-2018')
italian1_season18_19 = prepare_dataframe_italian1('I1_18_19.csv', '2018-2019')
italian1_season19_20 = prepare_dataframe_italian1('I1_19_20.csv', '2019-2020')
italian1_season20_21 = prepare_dataframe_italian1('I1_20_21.csv', '2020-2021')
italian1_season21_22 = prepare_dataframe_italian1('I1_21_22.csv', '2021-2022')
italian1_season22_23 = prepare_dataframe_italian1('I1_22_23.csv', '2022-2023')
italian1_season23_24 = prepare_dataframe_italian1('I1_23_24.csv', '2023-2024')
# italian1_season24_25 = prepare_dataframe_italian1('I1_24_25.csv', '2024-2025')

IndexError: list index out of range

In [69]:
italian1_data = pd.concat([italian1_season18_19, italian1_season19_20, italian1_season20_21, italian1_season21_22, italian1_season22_23, italian1_season23_24])
italian1_data.fillna(0, inplace = True)
italian1_data.to_csv('italian1_data.csv', index = False)

In [67]:
def prepare_dataframe_italian2(csv_file, season):
    season_data = pd.read_csv(csv_file)
    teams = season_data['HomeTeam'].unique()
    url = f"https://fbref.com/en/comps/18/{season}/schedule/{season}-Serie-B-Scores-and-Fixtures"
    season_data = join_scraped(url, season_data, season, teams, '18')
    season_data = clean_v2(season_data)
    season_data = create_seasonal_table(season_data,teams)
    season_data = calculate_odds(season_data)
    return season_data

In [75]:
# spanish1_season17_18 = prepare_dataframe_spanish1('SP1_17_18.csv', '2017-2018')
# italian2_season18_19 = prepare_dataframe_italian2('I2_18_19.csv', '2018-2019')
italian2_season19_20 = prepare_dataframe_italian2('I2_19_20.csv', '2019-2020')
italian2_season20_21 = prepare_dataframe_italian2('I2_20_21.csv', '2020-2021')
italian2_season21_22 = prepare_dataframe_italian2('I2_21_22.csv', '2021-2022')
italian2_season22_23 = prepare_dataframe_italian2('I2_22_23.csv', '2022-2023')
italian2_season23_24 = prepare_dataframe_italian2('I2_23_24.csv', '2023-2024')
# italian1_season24_25 = prepare_dataframe_italian1('I1_24_25.csv', '2024-2025')

In [77]:
italian2_data = pd.concat([italian1_season19_20, italian1_season20_21, italian1_season21_22, italian1_season22_23, italian1_season23_24])
italian2_data.fillna(0, inplace = True)
italian2_data.to_csv('italian2_data.csv', index = False)

In [79]:
def prepare_dataframe_german1(csv_file, season):
    season_data = pd.read_csv(csv_file)
    teams = season_data['HomeTeam'].unique()
    url = f"https://fbref.com/en/comps/20/{season}/schedule/{season}-Bundesliga-Scores-and-Fixtures"
    season_data = join_scraped(url, season_data, season, teams, '20')
    season_data = clean_v2(season_data)
    season_data = create_seasonal_table(season_data,teams)
    season_data = calculate_odds(season_data)
    return season_data

In [83]:
german1_season18_19 = prepare_dataframe_german1('D1_18_19.csv', '2018-2019')
german1_season19_20 = prepare_dataframe_german1('D1_19_20.csv', '2019-2020')
german1_season20_21 = prepare_dataframe_german1('D1_20_21.csv', '2020-2021')
german1_season21_22 = prepare_dataframe_german1('D1_21_22.csv', '2021-2022')
german1_season22_23 = prepare_dataframe_german1('D1_22_23.csv', '2022-2023')
german1_season23_24 = prepare_dataframe_german1('D1_23_24.csv', '2023-2024')
german1_season24_25 = prepare_dataframe_german1('D1_24_25.csv', '2024-2025')

In [84]:
german1_data = pd.concat([german1_season18_19, german1_season19_20, german1_season20_21, german1_season21_22, german1_season22_23, german1_season23_24,german1_season24_25])
german1_data.fillna(0, inplace = True)
german1_data.to_csv('german1_data.csv', index = False)

In [87]:
def prepare_dataframe_german2(csv_file, season):
    season_data = pd.read_csv(csv_file)
    teams = season_data['HomeTeam'].unique()
    url = f"https://fbref.com/en/comps/33/{season}/schedule/{season}-2Bundesliga-Scores-and-Fixtures"
    season_data = join_scraped(url, season_data, season, teams, '33')
    season_data = clean_v2(season_data)
    season_data = create_seasonal_table(season_data,teams)
    season_data = calculate_odds(season_data)
    return season_data

In [617]:
season_data0 = pd.read_csv('24_25_eng.csv')
season_data00 = pd.read_csv('23_24_eng.csv')
season_data2 = pd.read_csv('22_23_eng.csv')
season_data3 = pd.read_csv('21_22_eng.csv')
season_data4 = pd.read_csv('20_21_eng.csv')
season_data5 = pd.read_csv('19_20_eng.csv')
season_data6 = pd.read_csv('18_19_eng.csv')
season_data7 = pd.read_csv('17_18_eng.csv')

championship_data0 = pd.read_csv('championship_24_25.csv')
championship_data1 = pd.read_csv('championship_23_24.csv')
championship_data2 = pd.read_csv('championship_22_23.csv')
championship_data3 = pd.read_csv('championship_21_22.csv')
championship_data4 = pd.read_csv('championship_20_21.csv')
championship_data5 = pd.read_csv('championship_19_20.csv')
championship_data6 = pd.read_csv('championship_18_19.csv')

spanish1_data0 = pd.read_csv('SP_24_25.csv')
spanish1_data1 = pd.read_csv('SP1_23_24.csv')
spanish1_data2 = pd.read_csv('SP1_22_23.csv')
spanish1_data3 = pd.read_csv('SP1_21_22.csv')
spanish1_data4 = pd.read_csv('SP1_20_21.csv')
spanish1_data5 = pd.read_csv('SP1_19_20.csv')
spanish1_data6 = pd.read_csv('SP1_18_19.csv')

# spanish2_data0 = pd.read_csv('SP2_24_25.csv')
spanish2_data1 = pd.read_csv('SP2_23_24.csv')
spanish2_data2 = pd.read_csv('SP2_22_23.csv')
spanish2_data3 = pd.read_csv('SP2_21_22.csv')
spanish2_data4 = pd.read_csv('SP2_20_21.csv')
spanish2_data5 = pd.read_csv('SP2_19_20.csv')
spanish2_data6 = pd.read_csv('SP2_18_19.csv')

italian1_data1 = pd.read_csv('I1_23_24.csv')
italian1_data2 = pd.read_csv('I1_22_23.csv')
italian1_data3 = pd.read_csv('I1_21_22.csv')
italian1_data4 = pd.read_csv('I1_20_21.csv')
italian1_data5 = pd.read_csv('I1_19_20.csv')
italian1_data6 = pd.read_csv('I1_18_19.csv')

italian2_data1 = pd.read_csv('I2_23_24.csv')
italian2_data2 = pd.read_csv('I2_22_23.csv')
italian2_data3 = pd.read_csv('I2_21_22.csv')
italian2_data4 = pd.read_csv('I2_20_21.csv')
italian2_data5 = pd.read_csv('I2_19_20.csv')
# italian2_data6 = pd.read_csv('I2_18_19.csv')

german1_data0 = pd.read_csv('D1_24_25.csv')
german1_data1 = pd.read_csv('D1_23_24.csv')
german1_data2 = pd.read_csv('D1_22_23.csv')
german1_data3 = pd.read_csv('D1_21_22.csv')
german1_data4 = pd.read_csv('D1_20_21.csv')
german1_data5 = pd.read_csv('D1_19_20.csv')
german1_data6 = pd.read_csv('D1_18_19.csv')

german2_data0 = pd.read_csv('D2_24_25.csv')
german2_data1 = pd.read_csv('D2_23_24.csv')
german2_data2 = pd.read_csv('D2_22_23.csv')
german2_data3 = pd.read_csv('D2_21_22.csv')
german2_data4 = pd.read_csv('D2_20_21.csv')
german2_data5 = pd.read_csv('D2_19_20.csv')
german2_data6 = pd.read_csv('D2_18_19.csv')

season_data = pd.concat([season_data7, season_data6,season_data5,season_data4,season_data3,season_data2,season_data00,season_data0,
                        championship_data0, championship_data1, championship_data2, championship_data3, championship_data4, championship_data5, championship_data6,
                        spanish1_data0, spanish1_data1,spanish1_data2,spanish1_data3,spanish1_data4,spanish1_data5,spanish1_data6,
                          spanish2_data1,spanish2_data2,spanish2_data3,spanish2_data4,spanish2_data5,spanish2_data6,
                        italian1_data1,italian1_data2,italian1_data3,italian1_data4,italian1_data5,italian1_data6,
                         italian2_data2,italian2_data3,italian2_data4,italian2_data5,italian2_data1,
                         german1_data0, german1_data1,german1_data2,german1_data3,german1_data4,german1_data5,german1_data6,
                          german2_data0, german2_data1,german2_data2,german2_data3,german2_data4,german2_data5,german2_data6,
                        ])
season_data['Date'] = pd.to_datetime(season_data['Date'], format = "%d/%m/%Y")
season_data.sort_values(['Date'],ascending = True, inplace = True)
season_data.reset_index(drop = True, inplace = True)
season_data =  get_team_h2h(season_data)

In [623]:
season_data.to_csv('h2h_data.csv', index = False)

In [89]:
german2_season18_19 = prepare_dataframe_german2('D2_18_19.csv', '2018-2019')
german2_season19_20 = prepare_dataframe_german2('D2_19_20.csv', '2019-2020')
german2_season20_21 = prepare_dataframe_german2('D2_20_21.csv', '2020-2021')
german2_season21_22 = prepare_dataframe_german2('D2_21_22.csv', '2021-2022')
german2_season22_23 = prepare_dataframe_german2('D2_22_23.csv', '2022-2023')
german2_season23_24 = prepare_dataframe_german2('D2_23_24.csv', '2023-2024')
german2_season24_25 = prepare_dataframe_german2('D2_24_25.csv', '2024-2025')

In [90]:
german2_data = pd.concat([german2_season18_19, german2_season19_20, german2_season20_21, german2_season21_22, german2_season22_23, german2_season23_24,german2_season24_25])
german2_data.fillna(0, inplace = True)
german2_data.to_csv('german2_data.csv', index = False)